In [1]:
%pylab inline
import scipy.optimize
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

Populating the interactive namespace from numpy and matplotlib


## Data

In [2]:
def data2numpy(D):
    X = np.array([r[0] for r in D])
    y = np.array([r[1] for r in D])
    return X, y
# [([data points], [targets])]
data_AND = data2numpy([
    ([0, 0], [0]), 
    ([0, 1], [0]),
    ([1, 0], [0]),
    ([1, 1], [1]),
])

data_OR = data2numpy([
    ([0, 0], [0]), 
    ([0, 1], [1]),
    ([1, 0], [1]),
    ([1, 1], [1]),
])

data_XOR = data2numpy([
    ([0, 0], [0]), 
    ([0, 1], [1]),
    ([1, 0], [1]),
    ([1, 1], [0]),
])

print(data_AND)
print(data_OR)
print(data_XOR)

(array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]]), array([[0],
       [0],
       [0],
       [1]]))
(array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]]), array([[0],
       [1],
       [1],
       [1]]))
(array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]]), array([[0],
       [1],
       [1],
       [0]]))


## Network

In [45]:
class sigmoid:
    def f(z):
        return 1 / (1 + np.exp(-z))
    
    def df(z):
        s = sigmoid.f(z)
        return s * (1 - s)

class identity:
    def f(z):
        return z
    
    def df(z):
        return 1


class Layer(object):
    def __init__(self, inputs, neurons, activation_function, init_mult=0.9, alpha=0.001, regularization=.001):
        self.ci = inputs
        self.cn = neurons
        self.fce = activation_function
        self.W = np.random.rand(self.cn * self.ci) * init_mult
        self.W = self.W.reshape((self.cn, self.ci))
        self.b = np.random.rand(self.cn) * init_mult
        
        self.alpha = alpha
        self.regularization = regularization
        
    def ff(self, x):
        self.a1 = x
        self.z = x.dot(self.W.T)
        self.z += self.b
        self.a = self.fce.f(self.z)
        return self.a
    
    def bb(self, d):
        self.delta = d * self.fce.df(self.z)
        delta = self.delta.dot(self.W)
        return delta
    
    def gradient(self):
        return self.delta.T.dot(self.a1), self.delta.sum(axis=0)
    
    def update(self, m):
        gw, gb = self.gradient()
        self.W -= self.alpha * (gw / m + self.regularization * self.W)
        self.b -= self.alpha * gb / m
    
    def __mul__(self, x):
        return self.W * x
    
    def __rmul__(self, x):
        return x * self.W
    
    def dot(self, x):
        return self.W.dot(x)

    
class FFNN():
    def __init__(self, layers):
        self.L = layers
            
    def forward_pass(self, X):
        Xi = X
        for l in self.L:
            Xi = l.ff(Xi)
        return Xi
    
    def backprop(self, d):
        di = d
        for l in reversed(self.L):
            di = l.bb(di)
    def update(self, m):
        for l in self.L:
            l.update(m)
    
    def decrease_alpha(self):
        for l in self.L:
            l.alpha *= 0.1
    
    def fit(self, X, y, n_iters=10000000):
        oldd = np.inf
        for i in range(n_iters):
            yp = self.forward_pass(X)
            d = -(y - yp)
            e = (d ** 2).mean()
            if oldd < e:
                break
            
            self.backprop(d)
            self.update(X.shape[0])
            if i % 100000 == 0:
                print(i, e)
                print(self.L[-1].W)
                print(yp)
                print()
            oldd = e
                
        

# for X, y in [data_AND, data_OR, data_XOR]:    
for X, y in [data_XOR]:        
    layers = [
        Layer(2, 2, sigmoid),
        Layer(2, 1, sigmoid, alpha=0.0001)    
    ]
    nn = FFNN(layers)
    nn.fit(X, y)
    x=nn.forward_pass(X)
    print("result")
    print(((x-y)).mean(), nn.forward_pass(X))
    print()   
    

    
    

0 0.373153545703
[[ 0.87921904  0.57838073]]
[[ 0.83267177]
 [ 0.85403282]
 [ 0.85609302]
 [ 0.87020473]]

100000 0.277991835435
[[ 0.58154616  0.26513301]]
[[ 0.6538734 ]
 [ 0.66639223]
 [ 0.66963752]
 [ 0.68116328]]

200000 0.253291908335
[[ 0.45937454  0.1130109 ]]
[[ 0.55283985]
 [ 0.55613316]
 [ 0.55894391]
 [ 0.56212764]]

300000 0.250379204136
[[ 0.41966181  0.06066893]]
[[ 0.51785445]
 [ 0.51852994]
 [ 0.52105448]
 [ 0.52167169]]

400000 0.250037169852
[[ 0.40404454  0.04300412]]
[[ 0.50575046]
 [ 0.50563405]
 [ 0.50799159]
 [ 0.50783382]]

500000 0.249997517294
[[ 0.39625853  0.03722902]]
[[ 0.50147113]
 [ 0.5010914 ]
 [ 0.50332642]
 [ 0.50291385]]

600000 0.249994156535
[[ 0.3912176   0.03557996]]
[[ 0.49995179]
 [ 0.49947989]
 [ 0.50161059]
 [ 0.50111121]]

result
-0.000531587658633 [[ 0.49994573]
 [ 0.49947344]
 [ 0.50160345]
 [ 0.50110373]]



In [35]:
x= [[ 0.5001287 ],
 [ 0.50013034],
 [ 0.5001313 ],
 [ 0.50013294]]
-(x-y).mean()

-0.00013081999999997596

In [36]:
y

array([[0],
       [1],
       [1],
       [0]])

# Digits classification

## Load data

In [5]:
mnist = fetch_mldata('MNIST original', data_home='./data')
y_all = mnist.target[:, np.newaxis]
intercept = np.ones_like(y_all)
data = np.hstack([intercept, mnist.data, y_all])
np.random.shuffle(data)

In [6]:
def normalize_features(train, test):
    """Normalizes train set features to a standard normal distribution
    (zero mean and unit variance). The same procedure is then applied
    to the test set features.
    """
    train_mean = train.mean(axis=0)
    # +0.1 to avoid division by zero in this specific case
    train_std = train.std(axis=0) + 0.1
    
    train = (train - train_mean) / train_std
    test = (test - train_mean) / train_std
    return train, test

In [7]:
train_data_count = 60000
Xt = data[:train_data_count, :-1]
yt = data[:train_data_count, -1:].astype(int)

Xv = data[train_data_count:, :-1]
yv = data[train_data_count:, -1:].astype(int)

Xt, Xv = normalize_features(Xt, Xv)

## Softmax FNN

In [33]:
class FFNN():
    def __init__(self, layers_dims=[], activation_functions=[], regularization=1.1):
        self.layers_dims = layers_dims
        self.activation_functions = [None] + activation_functions
        weights_size = 0
        for i in range(1, len(self.layers_dims)):
            weights_size += self.layers_dims[i-1] * self.layers_dims[i]
        self.b_size = sum(layers_dims[1:])
        self.weights = np.random.rand(self.b_size + weights_size) * 0.0001
        print(self.weights.shape)
        self.training_iterations = 0
        self.lambd = regularization
    
    def next_iteration(self, weights):
        self.training_iterations += 1
        if self.training_iterations % 10 == 0:
            print("Iteration:", self.training_iterations, flush=True)
            yvp = nn.forward_pass(Xv)
            yvp = yvp.argmax(axis=1)
            print('Training accuracy: {}'.format(accuracy_score(yt, ytp)), flush=True)
    
    def cost(self, weights, X, y):
        nl = len(self.layers_dims) - 1
        deltas = [None for i in range(nl + 1)]
        
        self.weights = weights
        self.forward_pass(X, store_results=True)
        a = self.a
        z = self.z
        B, W = self.unfold_weights(weights)
        deltas[nl] = -(y - a[nl])
        deltas[nl] *= self.activation_functions[nl].df(z[nl])
        # deltas
        for l in range(nl, 1, -1):
            deltas[l-1] = deltas[l].dot(W[l]) * self.activation_functions[l-1].df(z[l-1])
        weights_derivatives = []
        b_derivatives = []
        
        # derivatives
        for l in range(1, nl+1):
            dW = deltas[l].T.dot(a[l-1])
            weights_derivatives.append(dW.flatten())
            b_derivatives.append(deltas[l].sum(axis=0).flatten())
        cost = ((y - a[nl]) ** 2).sum() + self.lambd * (weights ** 2).sum()
        gradient = np.hstack(b_derivatives + weights_derivatives)

        assert gradient.shape == weights.shape
        return cost, gradient
    
    def unfold_weights(self, weights):
        used_count = self.b_size
        used_bcount = 0
        W = [None]
        B = [None]
        for i in range(1, len(self.layers_dims)):
            m, n = self.layers_dims[i], self.layers_dims[i-1]
            w = self.weights[used_count:used_count + (m * n)]
            w = w.reshape((m, n))
            b = self.weights[used_bcount:used_bcount + m]
            W.append(w)
            B.append(b)
            used_count += m * n
            used_bcount += m
        return B, W
            
    def forward_pass(self, X, store_results=False):
        used_count = self.b_size
        used_bcount = 0
        Xl = X
        B, W = self.unfold_weights(self.weights)
        if store_results:
            self.a = [X]
            self.z = [X]
        for i in range(1, len(self.layers_dims)):
            zl = Xl.dot(W[i].T) + B[i]
            Xl = self.activation_functions[i].f(zl)
            if store_results:
                self.z.append(zl)
                self.a.append(Xl)
        return Xl 
    
    def fit(self, X, y):
        res = scipy.optimize.minimize(
            fun=self.cost,
            x0=self.weights,
            args=(X, y),
            method='L-BFGS-B',
            jac=True,
            tol=1e-5,
            options={'maxiter': 500, 'disp': True},
            callback=self.next_iteration,
        )
        self.weights = res.x



le = preprocessing.LabelBinarizer()
nn = FFNN([785, 256, 10], [sigmoid, sigmoid, sigmoid])
# train 
ytt = le.fit_transform(yt)
nn.fit(Xt, ytt)
ytp = nn.forward_pass(Xt)
ytp = ytp.argmax(axis=1)
# test
yvt = le.transform(yv)
yvp = nn.forward_pass(Xv)
yvp = yvp.argmax(axis=1)

print('Training accuracy: {}'.format(accuracy_score(yt, ytp)))
print('Test accuracy: {}'.format(accuracy_score(yv, yvp)))

(203786,)
Training accuracy: 0.1134
Test accuracy: 0.1073


In [24]:
ytp = nn.forward_pass(Xt)
ytp = ytp.argmax(axis=1)
# test
yvt = le.transform(yv)
yvp = nn.forward_pass(Xv)
yvp = yvp.argmax(axis=1)

print('Training accuracy: {}'.format(accuracy_score(yt, ytp)))
print('Test accuracy: {}'.format(accuracy_score(yv, yvp)))

Training accuracy: 0.70855
Test accuracy: 0.6982
